# Harmony Search Algorithm


**Decision Variables:**
- $x_{ij}$: Quantity of goods to transport from supplier $i$ to customer $j$, for all suppliers $i \in \{1, 2, \ldots, n\}$ and customers $j \in \{1, 2, \ldots, m\}$.
- $y_{i}$: Binary decision variable indicating whether supplier $i$ is active (1) or not (0), for all suppliers $i \in \{1, 2, \ldots, n\}$.

**Parameters:**
- $C_{ij}$: Transportation cost per unit from supplier $i$ to customer $j$.
- $F_{i}$: Fixed cost of activating supplier $i$.
- $D_{j}$: Demand of customer $j$.
- $S_{i}$: Supply capacity of supplier $i$.

**Objective Function:**
Minimize the total cost, which includes the transportation costs from suppliers to customers and the fixed costs of activating suppliers:
$$\text{Minimize} \quad Z = \sum_{i=1}^{n} F_{i} y_{i} + \sum_{i=1}^{n}\sum_{j=1}^{m} C_{ij} x_{ij}$$

**Constraints:**
1. Supply Constraint: The total goods supplied by each supplier cannot exceed their capacity:
   $$\sum_{j=1}^{m} x_{ij} \leq S_{i} \cdot y_{i}, \quad \forall i \in \{1, 2, \ldots, n\}$$
2. Demand Satisfaction: Each customer zone's demand must be met by the shipments from the suppliers:
   $$\sum_{i=1}^{n} x_{ij} = D_{j}, \quad \forall j \in \{1, 2, \ldots, m\}$$
3. Non-negativity and Binary Constraints: The quantities shipped must be non-negative, and the decision to activate a supplier is binary:
   $$x_{ij} \geq 0, \quad \forall i \in \{1, 2, \ldots, n\}, j \in \{1, 2, \ldots, m\}$$
   $$y_{i} \in \{0, 1\}, \quad \forall i \in \{1, 2, \ldots, n\}$$


In [1]:
import numpy as np
import random

# Example data for the supply chain problem
C = np.array([[2, 3, 1.5, 4],   # Transportation costs
              [4, 1, 2.5, 3],
              [3, 2, 2, 2]])

F = np.array([5000, 4000, 4500])  # Fixed costs
D = np.array([300, 200, 150, 250])  # Demand
S = np.array([500, 300, 400])  # Supply

num_suppliers = len(S)
num_customers = len(D)

# Harmony Search parameters
num_harmonies = 100
max_iter = 1000
hmcr = 0.9  # Harmony Memory Considering Rate
par = 0.3   # Pitch Adjusting Rate
bw = 0.01   # Bandwidth

# Initialize Harmony Memory
harmony_memory = np.random.rand(num_harmonies, num_suppliers * num_customers + num_suppliers)

# Objective function
def objective_function(x):
    x_matrix = x[:num_suppliers * num_customers].reshape(num_suppliers, num_customers)
    y_vector = x[num_suppliers * num_customers:]
    transport_cost = np.sum(C * x_matrix)
    fixed_cost = np.sum(F * y_vector)
    return transport_cost + fixed_cost

# Check constraints
def check_constraints(x):
    x_matrix = x[:num_suppliers * num_customers].reshape(num_suppliers, num_customers)
    y_vector = x[num_suppliers * num_customers:]
    supply_constraint = np.all(np.sum(x_matrix, axis=1) <= S * y_vector)
    demand_constraint = np.all(np.sum(x_matrix, axis=0) == D)
    return supply_constraint and demand_constraint

# Harmony Search Algorithm
for iteration in range(max_iter):
    new_harmony = np.zeros(num_suppliers * num_customers + num_suppliers)
    
    # Memory consideration and pitch adjustment
    for i in range(num_suppliers * num_customers + num_suppliers):
        if random.random() < hmcr:
            new_harmony[i] = random.choice(harmony_memory[:, i])
            if random.random() < par:
                new_harmony[i] += bw * (random.random() - 0.5)
        else:
            new_harmony[i] = random.random()
    
    # Check if new harmony satisfies constraints
    if check_constraints(new_harmony):
        # Update Harmony Memory
        worst_harmony_index = np.argmax([objective_function(h) for h in harmony_memory])
        if objective_function(new_harmony) < objective_function(harmony_memory[worst_harmony_index]):
            harmony_memory[worst_harmony_index] = new_harmony

# Find the best solution
best_solution = min(harmony_memory, key=objective_function)
best_cost = objective_function(best_solution)

print("Best Solution:", best_solution)
print("Best Cost:", best_cost)


Best Solution: [0.35706263 0.34836246 0.63669454 0.88823324 0.87222196 0.34006378
 0.40635209 0.80635501 0.74586315 0.20386664 0.02360274 0.11750397
 0.04680393 0.15501418 0.03489613]
Best Cost: 1027.5675860710162
